# Imports

In [ ]:
# Add imports
from python import capacity, lecturesperfaculty, faculty, lectureinf, geo, rolli, workload, schedulemap
import pandas
import plotly.express
import sqlalchemy

db = sqlalchemy.create_engine('sqlite:///../../Daten/univis.db')

# 0. Vorbereitung

Zuerst mussten wir die für die Moduldatenbank und das UnivIS Parser schreiben, damit wir besser mit den Daten arbeiten können.

**Parser für die Moduldatenbank**
Zu finden ist der Parser in dem Modul `moduldbparser.py`.
Wir nutzen eine XML-Datei, die alle Module des Modulinformationssystems Informatik enthält und auf der Seite der Moduldatenbank erhältlich ist. In dieser XML-Datei ist zwar jedes Modul aufgelistet, aber um die Informationen zu den Modulen zu erhalten, muss eine URL aufgerufen werden, die in der XML-Datei vorhandenen ist. Ruft man diese URL auf, öffnet sich eine weitere XML-Datei.

Um nun diese Informationen darzustellen, muss die Funktion `get_dataframe(file_name)` mit der ursprünglichen XML-Datei aufgerufen werden. Es wird ein Dictionary daraus erstellt, jedes Modul durchgegangen, die URLs aufgerufen, die erhaltene XML-Datei mit der Funktion `parse_url(content, dict_modules)` in ein Dictionary umgewandelt, die Zahlen in integers umgewandelt und die Wörterbücher in separate Key-Value-Paare umgewandelt. Schlussendlich kann ein pandas Dataframe erstellt werden.

**Parser für UnivIS**
Eine ausführliche Beschreibung befindet sich in der Datei univis.ipynb.


# 1. Daten sammeln und aufbereiten

## In welchem Maße wird die Kapazität der Räume genutzt?

Die benötigten Daten werden mit der Funktion `capacity(db)` aus dem Modul `capacity.py` gesammelt. Da es so viele verschiedene Räume gibt, haben wir uns entschieden, die Räume nach Größe aufzuteilen, um die Graphen übersichtlich zu halten. Wir sammeln die Daten für Räume mit einer Kapazität von mindestens 100 Personen, für Räume mit einer Kapazität von mindestens 50 aber höchstens 99 Personen und für Räume mit einer Kapazität von höchstens 49 Personen. Außerdem wollen wir noch den Mittelwert der Raumauslastung für jede Raumgröße und jedes Semester darstellen.

In [ ]:
rooms_from_100, rooms_til_100, rooms_til_50, mean_f100, mean_t100, mean_t50 = capacity.capacity(db)

In der SQL-Anfrage wird die durchschnittliche Raumauslastung jedes Raums je Semester berechnet. Dafür nutzen wir die erwartete Teilnehmerzahl der Veranstaltungen, die in diesem Raum stattfinden. Je nach Kapazität werden die Informationen zum Raum in einem anderen Dictionary abgespeichert. Um den Mittelwert herauszufinden, wird für jede Raumgröße die Raumauslastungen der entsprechenden Räume addiert und am Ende diese Summe durch die Anzahl der Räume geteilt.

Als Nächstes müssen diese Dictionaries in Dataframes umgewandelt werden.

In [ ]:
df_f_100 = capacity.get_dataframe(rooms_from_100)
df_t_100 = capacity.get_dataframe(rooms_til_100)
df_t_50 = capacity.get_dataframe(rooms_til_50)

Zunächst wird das Dictionary jeweils in ein Dataframe umgewandelt. Jedoch ist jeder Eintrag der Liste bzw. jedes Dictionary, das die Informationen zu den Räumen enthält, in einer extra Spalte. Deswegen muss über die Spalten iteriert werden, damit die Spalte bzw. das Dictionary in mehrere Spalten aufgeteilt wird, wobei der Key der neue Spaltenname wird. Dabei entsteht jedoch pro aufgeteilte Spalte ein neues Dataframe. Die so erzeugten Dataframes müssen zusammengefügt werden. Danach wird das neue Dataframe noch so angepasst, dass wir damit unsere Graphen erstellen können.

Die Dictionaries zum Mittelwert der Raumgrößen müssen in ein gemeinsames Dataframe umgewandelt werden.

In [ ]:
df_means = capacity.get_dataframe_for_mean(mean_f100, mean_t100, mean_t50)

Es werden die Dictionaries jeweils in Dataframes umgewandelt, Spalten und Reihen vertauscht, die Spaltennamen angepasst und am Ende die Dataframes zusammengefügt.

## Wie sehr wurde die Kapazität während Corona verringert?

Die benötigten Daten werden mit der Funktion `capacity_corona(db)` aus dem Modul `capacity.py` gesammelt. Da es so viele verschiedene Räume gibt, haben wir uns entschieden die Räume nach Größe aufzuteilen, um die Graphen übersichtlich zu halten, s.o.

In [ ]:
corona_from_100, corona_til_100, corona_til_50 = capacity.capacity(db)

In der SQL-Anfrage wird die Kapazität während Corona im Verhältnis zur sonstigen Kapazität gesetzt und gleichzeitig pro Raum die Auslastung berechnet. Je nach Kapazität werden die Informationen zum Raum in dem entsprechenden Dictionary abgespeichert.

Als Nächstes müssen diese Dictionaries in Dataframes umgewandelt werden, s.o.

In [ ]:
df_cf_100 = capacity.get_dataframe(corona_from_100)
df_ct_100 = capacity.get_dataframe(corona_til_100)
df_ct_50 = capacity.get_dataframe(corona_til_50)

## In welchem Maße wurde die reduzierte Kapazität während Corona genutzt?

Die Daten werden wie in der vorigen Frage gesammelt und aufbereitet. Dies ist möglich, da in der SQL-Anfrage ebenfalls die durchschnittliche Raumauslastung während Corona berechnet wird.

## Wie hat sich die Raumverteilung der Fakultäten im Laufe der Zeit verändert?

Um diese Frage zu beantworten, müssen wir erst einmal die Veranstaltungen nach Fakultäten und Semester sortieren, zu den Veranstaltungen die Räume heraussuchen, die zugehörigen Adressen bestimmen und daraufhin die Koordinaten der Adressen herausfinden.
Da die Adressen im UnivIS nicht einheitlich gespeichert und teilweise falsch geschrieben sind, mussten wir zunächst die Adressen parsen, um Koordinaten herauszufinden. Das Ergebnis speichern wir in einer Tabelle in unserer SQL-Datenbank, die durch den Aufruf der Funktion `create_table(db)` aus dem Modul `sqladdr.py`  erstellt und durch die Funktion `create_adress(db)` mit Einträgen gefüllt wird. Dabei ruft die Funktion `new_addr(addr)` aus dem Modul `parse_addr.py` auf, wodurch vor allem Abkürzungen aufgelöst und Rechtschreibfehler korrigiert werden.

Nun kann mit der Funktion `addr_per_faculty(db, geoman: GeoManager)` aus dem Modul `lecturesperfaculty.py` die oben beschriebenen Informationen gesammelt werden:

In [ ]:
gm = lecturesperfaculty.GeoManager("stu207503@mail.uni-kiel.de")
all_coords = lecturesperfaculty.addr_per_faculty_now(db, gm)

Durch die SQL-Anfrage werden alle Räume, in denen Module stattgefunden haben, und auch direkt die geparsten Adressen der Räume bestimmt. Diese Räume werden nach Semester, Fakultät und Raum gruppiert und nach Semester sortiert. Nun wird jeder Eintrag des Ergebnisses durchgegangen. Dabei werden zu unserer geparsten Adresse die Koordinaten mithilfe der Klasse `Geomanger` aus dem Modul `geomanager.py` bestimmt. Dies ist eine Klasse, die einen Cache für die Zuordnung von Adressen und Koordinaten verwaltet. Wenn eine noch unbekannte Adresse angefragt wird, leitet der Geomanager die Anfrage gemäß den [terms of service](https://operations.osmfoundation.org/policies/nominatim/) an Nominatim weiter, ansonsten gibt er den gecachten Wert zurück. Der Cache kann außerdem als json importiert/exportiert werden.
 Schlussendlich werden die Informationen zum Raum in ein Dictionary gespeichert.

## Welche Wege müssen Studierende der Informatik bzw. Wirtschaftsinformatik in einem Semester gehen?

Wir analysieren hier nur die Bachelorstudiengänge, da hier genügend Pflichtmodule vorgeschrieben sind, um aussagekräftige Daten zu erhalten. Außerdem betrachten wir nur die Fachprüfungsordnungen ab 2015, da vorher keine Studienverlaufspläne in den FPOs zu finden sind.

Wir müssen zunächst die Vorlesungen, Übungen, Seminare usw. sammeln, die in einem Semester gehört werden. Für Informatik beschränken wir uns auf die Pflichtmodule und für Wirtschaftsinformatik wird die BWL-Variante des Studiengangs ohne die Wahlpflichtmodule betrachtet. Jedoch müssen wir hier auch darauf achten, dass die Veranstaltungen dem richtigen Fachsemester zugeordnet werden. Die Schwierigkeit ist die Veranstaltungen der aktiven Fachprüfungsordnungen zuzurechnen.

Mithilfe der Funktion `build_schedule(db, modules, winf_modules)` aus dem Modul `lecturesperfaculty.py` werden zuerst die Lectures einem Fachsemester und Semester zugeordnet.
Dafür müssen jedoch vorher die Lectures gesammelt werden, die für Informatik bzw. Wirtschaftsinformatik relevant sein könnten. Zuerst werden mithilfe der Funktion `get_lectures(db)` alle Lectures aus dem UnivIS den Fakultäten und Semestern zugeordnet.

In [ ]:
all_modules = lecturesperfaculty.get_lectures(db)

Für Informatik brauchen wir die Module der Technischen Fakultät und der
Mathematisch-Naturwissenschaftlichen Fakultät:

In [ ]:
techn = all_modules[faculty.Faculty.TECHN.value]
mathe = all_modules[faculty.Faculty.MATHE.value]

modules_inf = {}
for sem in techn:
    modules_inf[sem] = techn[sem] + mathe[sem]

Für Wirtschaftsinformatik brauchen wir zusätzlich die Module der Wirtschafts- und Sozialwissenschaftlichen Fakultät und Rechtswissenschaftlichen Fakultät:

In [ ]:
wirtsc = all_modules[lecturesperfaculty.Faculty.WIRTSC.value]
law = all_modules[lecturesperfaculty.Faculty.RECHTS.value]

modules_winf = {}
for sem in techn:
    modules_winf[sem] = techn[sem] + mathe[sem] + wirtsc[sem] + law[sem]

Nun kann die Funktion `build_schedule(db, modules, winf_modules)` aufgerufen werden.

Für Informatik:

In [ ]:
schedule_ready_inf = lectureinf.build_schedule(db, modules_inf)

Für Wirtschaftsinformatik:

In [ ]:
schedule_ready_winf = lectureinf.build_schedule(db, modules_winf, True)

Dabei wird je nachdem, ob die Funktion für Wirtschaftsinformatik oder Informatik aufgerufen wurde, das entsprechende Dictionary mit den Pflichtmodulen der verschiedenen FPOs genutzt.
Nun wird mithilfe der SQL-Anfrage für jedes Semester die Lectures bestimmt. Durch das Ergebnis der Anfrage wird iteriert (jeder Eintrag stellt ein Modul dar), wobei zunächst die Module der technischen Fakultät aussortiert werden, die nicht zum Institut der Informatik und nicht zu den gefragten Studiengängen gehören. Daraufhin werden die verschiedenen FPOs eines Studiengangs und die Fachsemester der aktuellen FPO durchgegangen. Als Nächstes wird eine Formel angewendet, um zu sehen, ob die aktuelle FPO in dem aktuellen Semester und Fachsemester aktiv ist. Wenn dies der Fall ist, kann überprüft werden, ob die momentane Veranstaltung für das aktuelle Fachsemester und die FPO vorgesehen ist.

Einige Module müssen gesondert behandelt werden:
    -	Seminare und Projekte werden in einem Semester mehrfach angeboten, aber wir wollen jeweils nur eins aufnehmen
    -	Module für Zweifächler sollen ignoriert werden
    -	Statistische Methoden und Wissenschaftliches Arbeiten werden auch von anderen Fakultäten angeboten
    -	TGI für Wirtschaftsinformatiker hat in manchen Semestern einen eigenen Eintrag, aber in anderen wird der UnivIS-Eintrag von den Informatikern übernommen
    -	Die Vorlesung „Einführung in die Volkswirtschaftslehre“ für Wirtschafsinformatiker soll nur einmal aufgenommen werden

Ist diese Funktionen fertig, fehlen nun noch die Übungen. Dabei muss darauf geachtet werden, dass die Übungen in den gleichen Fachsemestern aufgenommen werden wie die Vorlesungen. Für diese Aufgabe wurde die Funktion `add_exercises_inf(schedule, db)` bzw. `add_exercises_winf(schedule, db)` geschrieben, die im Modul `lectureinf.py` zu finden ist.


Für Informatik:

In [ ]:
schedule_exercises_inf = lectureinf.add_exercises_inf(schedule_ready_inf, db)

Für Wirtschaftinformatik:

In [ ]:
schedule_exercises_winf = lectureinf.add_exercises_winf(schedule_ready_winf, db)

In diesen Funktionen werden die Übungen hinzugefügt. Durch die UnivIS-Flag „parent-lv“ kann eine Übung der Vorlesung zugeordnet werden.
Für Wirtschaftsinformatik müssen einige Übungen gesondert behandelt werden. Zwei Übungen werden jeweils doppelt hinzugefügt. Die einzig andere Möglichkeit diesen Fehler zu beheben wäre nach der Flag „classification“ zu gruppieren, da die Übungen nur diesen Eintrag gemeinsam haben. Jedoch werden dadurch andere Übungen ebenfalls nicht aufgenommen, die jedoch für das Semester vorgesehen sind.
Eine andere Übung wird gar nicht erst aufgenommen, da der Eintrag zu parent-lv in der vorherigen Funktion nicht ins Ergebnis eingetragen wurde, was daran liegt, dass die Vorlesung mehrfach angeboten wird und im UnivIS eingetragen wurde. Deshalb muss die Übung am Ende der Funktion extra aufgenommen.

Somit haben wir die Vorlesungen, Übungen, Seminare usw. den Fachsemestern eines Semesters zugeordnet.
Nun kann ein beispielhafter Stundenplan für jedes Semester und Fachsemester entwickelt werden. Dazu wird die Funktion `get_dependencies(db, schedule)` verwendet.

Für Informatik:

In [ ]:
plan_inf = lectureinf.get_dependencies(schedule_exercieses_inf, db)

Für Wirtschaftsinformatik:

In [ ]:
plan_winf = lectureinf.get_dependencies(schedule_exercieses_winf, db)

Die Funktion `get_dependencies` erstellt mithilfe der Datenbank und einer Auflistung an Pflichtmodulen pro Semester und Fachsemester (siehe oben) einen exemplarischen Stundenplan für alle gegebenen Semester und Fachsemester. Dabei werden Vorlesungen als verplfichtend angesehen, Übungen gelten für das Programm als vernachlässigbar. Zudem wird dem Stundenplan maximal ein Übungstermin pro Modul hinzugefügt. Wenn verschiedene Vorlesungen zum selben Zeitpunkt stattfinden, kann kein valider Stundenplan generiert werden. get_dependencies nutzt die Funktion `example_schedule`. Diese baut den Stundenplan eines einzelnen Semesters und Fachsemesters rekursiv auf.
Im Studiengang Wirtschaftsinformatik können für einige Semester keine validen Stundepläne gefunden werden, da sich in diesen Fällen für einige Fachsemester Vorlesungen von Pflichtmodulen überschneiden.

| Semester | Fachsemester | Überschnitt |
| -------- | ------------ | ----------- |
| 2016s    | 2.           | ADS und Produkion & Logistik |
| 2021s    | 4.           | TGI und Entscheidungsrechnungen |
| 2022s    | 2.           | ProgOO und Entscheidungsrechnungen |
| 2022s    | 4.           | TGI und Entscheidungsrechnungen |

Am Ende erhalten wir die Adressen der Räume, in denen die Lectures des Zeitplans stattfinden.

Haben wir nun die Adressen, müssen für die Darstellung noch die Koordinaten gesammelt werden. Dafür ist die Funktion `info_coords(dependencies)` aus dem Modul `geo.py` zuständig.

Für Informatik:

In [ ]:
coors_inf = geo.inf_coords(plan_inf)

Für Wirtschaftsinformatik:

In [ ]:
coors_winf = geo.inf_coords(plan_winf)

Es wird durch den vorher erstellten Zeitplan iteriert und die Koordinaten im Geomanager angefragt. Um die Darstellung der Wege möglichst genau zu halten, werden nur Adressen überprüft, die eine Hausnummer beinhalten, da ansonsten ein zufälliger Ort in der jeweiligen Straße genommen wird.
Wenn es zu der Adresse Koordinaten gibt, speichern wir sie im Dictionary.

Als Nächstes müssen nun die Routen berechnet werden, wofür die Funktion `routes_for_df(coor_dependencies)` verwendet wird.

Für Informatik:

In [ ]:
routes_inf = geo.routes_for_df(coors_inf)

Für Wirtschaftsinformatik:

In [ ]:
routes_winf = geo.routes_for_df(coors_winf)

Zunächst wird durch die Koordinaten iteriert. Wenn es zu einem Tag nur eine Koordinate gibt, wird diese direkt im Ergebnis gespeichert und der nächste Tag überprüft. Ansonsten wird über die Koordinaten iteriert, wobei nur die Route berechnet wird, wenn die aktuelle und nächste Koordinate nicht None ist und wir nicht die letzte Koordinate betrachten. In diesen Fällen wird die aktuelle Koordinate nur als Wegpunkt gespeichert. Andernfalls wird die Route aus dem Cache dieser Funktion übernommen oder mithilfe der Funktion `pyroutedistance(lon0, lat0, lon3, lat3)` aus dem Modul `osmroute.py` die Route bestimmt. In dieser Funktion wird das externe Modul `pyroutelib3` genutzt, um die Koordinaten zwischen zwei Punkten herauszufinden.

Der letzte Schritt ist die Routen in ein Dataframe umzuwandeln, damit es leichter ist, diese auf einer Karte dazustellen. Dies geschieht mit der Funktion `get_dataframe(routes)` aus dem Modul `geo.py`, die so ähnlich funktioniert wie die Funktion aus dem Modul `capacity.py`.

Für Informatik:

In [ ]:
fertig_inf = geo.get_dataframe(routes_inf)

Für Wirtschaftsinformatik:

In [ ]:
fertig_winf = geo.get_dataframe(routes_winf)

## Wie barrierefrei sind die Veranstaltungen der einzelnen Fakultäten für Rollstuhlfahrende?

Um die Daten für diese Frage zu erhalten, muss die Funktion `rollis(db)` aus dem Modul `rolli.py` verwendet werden.

In [ ]:
rolli_data = rolli.rollis(db)

Diese Funktion ermittelt für jede Fakultät und Semester, wie viele Räume für Rollstuhlfahrer zugänglich sind, und berechnet gleichzeitig den prozentualen Anteil.

Als Nächstes muss das Dictionary in ein Dataframe umgewandelt werden, v.a. müssen aus den Semestern und den Werten aus der Zelle zusammen mit der Fakultät eine Reihe entstehen:

In [ ]:
# Mit from_dict ein Dataframe erstellen
df = pd.DataFrame().from_dict(rolli_data)
# Den Index als Spalte rausholen
df = df.reset_index(level=0)
# Den Eelevanten Wert aus den Tupeln holen
df = df.applymap(lambda x: x[2] if type(x) is tuple and len(x) >= 3 else x)
# Semester als Spalten machen
df = pd.melt(df, id_vars=['index'])
# Die neu erstellten Spalten umbenennen
df = df.rename(columns={'index': 'Fakultät', 'variable': 'Semester', 'value': 'Prozentualer Anteil'})

## Wie hat sich die Arbeitsbelastung einer Lehrperson bezüglich Lehrveranstaltungen entwickelt?

Wir haben uns dazu entschieden nur die eigenständigen Veranstaltungen zu beachten, da zum Beispiel Übungen und Praktika von HiWis übernommen werden, die im UnivIS häufig nicht aufgeführt werden.
Um die Daten für diese Fragen zu erhalten, muss die Funktion `workloads(db)` aus dem Modul `workload.py` ausgeführt werden.

In [ ]:
df_work = workload.workloads(db)

In einer SQL-Anfrage wird die durchschnittliche Arbeitsbelastung einer Lehrperson für jede Fakultät und jedes Semester berechnet und aus der SQL-Query wird direkt ein Dataframe erstellt.

## In welcher Fakultät finden die meisten Veranstaltungen auf Englisch statt?

Wir haben uns dazu entschieden nur die eigenständigen Veranstaltungen zu beachten, da die Englisch-Flag für die Übungen usw. nicht gut gepflegt wird und deren Sprachen sich häufig an den übergeordneten Veranstaltungen orientieren.

Die benötigten Daten werden mit der Funktion `englishs(db)` aus dem Modul `english.py` gesammelt.

In [ ]:
df_english = english.englishs(db)

In einer SQL-Anfrage wird der prozentuale Anteil der englischsprachigen Lectures für jede Fakultät und jedes Semester berechnet und aus der SQL-Query wird direkt ein Dataframe erstellt.

## Wie hat sich der Anteil von Mitarbeitenden mit nicht-männlich erkannten Vornamen entwickelt?

## Gibt es Kriterien, anhand derer der Gewinner des Best-Prof-Awards vorhergesagt werden kann?

# 2. Daten visualisieren

## In welchem Maße wird die Kapazität eines Raumes genutzt?

Die Graphen werden alle durch die Funktion `visualize_capacity(df_rooms_f100, df_rooms_t100, df_rooms_t50, df_means)` erstellt:

In [ ]:
fig_means, fig_f100, fig_t100, fig_t50, fig_t50_part = capacity.visualize_capacity(df_f_100, df_t_100, df_t_50, df_means)

Um die Veränderung der Raumauslastungen aller Raumgrößen auf einen Blick zu sehen, erstellen wir einen Graphen, der den Mittelwert jedes Raumes für jedes Semester anzeigt:

In [ ]:
fig_means

Für die Räume mit einer Kapazität von mindestens 100 Personen bzw. von höchstens 100 Personen erstellen wir einen animierten Scatterplot:

In [ ]:
fig_f100

In [ ]:
fig_t100

 Für die kleineren Räume haben wir uns entschieden zwei einfache Scatterplots zu erstellen, da eine Animation wegen der vielen Räume und Daten unübersichtlich wird:

In [ ]:
fig_t50

In [ ]:
fig_t50_part

In dem Plot für die Räume mit einer Kapazität von mindestens 100 Personen ist zu erkennen, dass im Sommersemester 2021 die Raumauslastung aller Räume auf einmal konsequent sinkt. Dies scheint mit Corona zusammenzuhängen. Sonst ist auffällig, dass die Raumauslastung meistens unter 100% bleibt.
Es sticht heraus, dass mehrere mittlere Räume seit einigen Jahren im Wintersemester stark überbelegt wurden.
Bei den kleinen Räumen sind Häufungen z.B. bei einer Raumauslastung von 50% sichtbar. Unter 50% sind im Vergleich eher weniger Räume belegt. Dies erweckt den Eindruck, dass darauf geachtet wird, die kleinen Räume mit mindestens 50% zu belegen.

Wenn man sich die Mittelwerte anschaut, fällt auf, dass die größeren Räume die geringste und die kleineren Räume die höchste Raumauslastung aufweisen. Dabei bewegt sich die durchschnittliche Raumauslastung der mittleren und größeren Räume immer unter 100%, was darauf schließen lässt, dass diese Räume ihre Kapazitäten im Durchschnitt nicht vollständig ausschöpfen. Weiter ist erkennbar, dass die kleineren Räume im Vergleich zu den anderen Räumen stark bis überbelegt werden.

## Wie sehr wurde die Kapazität während Corona verringert?

Um diese Frage zu beantworten, haben wir für jede der drei Raumgrößen einen Scatterplot erstellt, der jeweils durch den Aufruf der Funktion `visualize_corona(df_corona_f100, df_corona_t100, df_corona_t50)` generiert werden.

In [ ]:
fig_cap_f100, fig_cap_t100, fig_cap_t50, fig_deg_f100, fig_deg_t100, fig_deg_t50 = capacity.visualize_corona(df_cf_100, df_ct_100, df_ct_50)

In [ ]:
fig_cap_f100

In [ ]:
fig_cap_t100

In [ ]:
fig_cap_t50

Im Zuge der Pandemie wurde die Kapazität der großen Räume in der Regel auf ca. 10% bis 18% der ursprünglichen Kapzität reduziert. Bei den mittleren Räumen wurde die Kapazität nicht ganz so stark verringert, aber befindet sich immer noch im Bereich von unter 50%. Die Kapazität der kleinen Räume wurde auf ca. 14% bis 70% reduziert, was deutlich über den Anteil der anderen Raumgrößen liegt.

## In welchem Maße wurde die reduzierte Kapazität während Corona genutzt?

Um diese Frage zu beantworten, haben wir für jede der drei Raumgrößen einen Scatterplot erstellt.

In [ ]:
fig_deg_f100

In [ ]:
fig_deg_t100

In [ ]:
fig_deg_t50

Es fällt auf, dass bei den größeren Räumen im Sommersemester 2020 stark darauf geachtet wurde, die Räume mit weniger Personen zu belegen. Aber in den folgenden Semestern ist die Raumauslastung je nach Raum sehr unterschiedlich und wieder stark angestiegen. Die mittleren Räume wurden während Corona allgemein nicht stark belastet und die kleineren Räume wurden wieder teilweise überbelegt.

## Wie hat sich die Raumverteilung der Fakultäten im Laufe der Zeit verändert?

Die Raumverteilung stellen wir auf einer Karte dar, wobei man die Zuordnung zu den verschiedenen Fakultäten sehen kann. Als weitere Art der Visualisierung haben wir uns für eine Heatmap entschieden. Diese zeigt die "Raumwanderung" der Fakultäten über die Semester an. Somit ist gut zu erkennen, an welchen Orten die verschiedenen Fakultäten die meisten Veranstaltungen haben.

In [ ]:
facultymap.create_map(DB, GM)

In [ ]:
facultymap.create_heatmap(DB, GM)['all']

Die meisten Veranstaltungen finden in Kiel und Umgebung statt, jedoch gibt es auch Ausnahmen, wie zum Beispiel im Jahr 2008 die Agrar- und Ernährungswissenschaftliche Fakultät, die jährlich eine Veranstaltung in Gartersleben (Sachsen-Anhalt) angeboten hat.
Die Module der Philosophischen und Rechtswissenschaftlichen Fakultäten finden auf dem gesamten Campus statt. Die Theologen hingegen sind sehr sesshaft in der Leibnizstraße 4.
An beiden Darstellungen erkennt man gut, wenn neue Gebäude eröffnet wurden. Zum Beispiel finden seit dem Wintersemester 2021 vermehrt Veranstaltungen der Rechtswissenschaftlichen Fakultät im Juridicum statt.

## Welche Wege müssen Studierende der Informatik bzw. Wirtschaftsinformatik in einem Semester gehen?

Für die Visualisierung dieser Fragen haben wir uns entschieden Karten zu erstellen. Diese sind auf der Website zu finden, da wir für Informatik und Wirtschaftsinformatik jeweils für jedes Semester seit dem Wintersemester 2015 erstellen. Um weitere Details zu erkennen, ist es ebenfalls möglich eine Karte für ein bestimmtes Fachsemester auszuwählen.

Als Beispiel ist hier eine Karte, die die Wege eines Informatikstudierenden im Wintersemester 2022/23 darstellt:

In [ ]:
schedulemap.inf_map(fertig_winf, "2022w")

Hier erklären wir kurz, wie wir diese Karten erstellen.
Ein Dataframe mit den entsprechenden Informationen, ein Semester und wenn gewünscht ein Fachsemester werden an die Funktion `inf_map(map_df, sem)` aus dem Modul `schedulemap.py` weitergegeben. Diese erstellen nun eine Karte mit einem Startpunkt auf dem Universitätsgelände und iterieren durch die Wochentage, wobei für jeden Wochentag ein Layer erstellt wird. Wenn nun durch das Dataframe iteriert wird, werden die Reihen übersprungen, die nicht zum aktuellen Semester, Tag und/oder Fachsemester gehören. Für jede Route in unserer Reihe wird eine Linie auf der Karte erstellt und für jeden Wegpunkt ein Marker gesetzt. So entsteht eine Karte.

An diesen Karten konnten wir erkennen, dass die meisten Veranstaltungen in dem Bereich um der Ludewig-Meyn-Straße und dem Christian-Albrechts-Platz stattfinden und die Studenten eher kurze Wege beschreiten müssen. Hauptsächlich die Studierenden aus den ersten drei Fachsemestern müssen Richtung Leibnizstraße gehen.
Im Gegensazu zu Informatikern haben die Wirtschaftsinformaitker auch im sechsten Fachsemester Pflictmodule.

## Wie barrierefrei sind die Veranstaltungen der einzelnen Fakultäten für Rollstuhlfahrende?

Für die Visualisierung erstellen wir einen Lineplot, der den Anteil an Räumen anzeigt, die mit einem Rollstuhl zugänglich sind.

In [ ]:
fig_r = plotly.express.line(df, x='Semester', y='Prozentualer Anteil', color='Fakultät', title='Rolli',
              color_discrete_map=FACULTY_COLORS)
fig_r.update_traces(hovertemplate=None)
fig_r.update_layout(hovermode='x')

Es ist auffällig, dass der Anteil an Räumen, die mit einem Rollstuhl zugänglich sind, bei der Theologischen Fakultäten ab dem Sommersemester 2005 stetig gestiegen und im Wintersemester 2016 plötzlich gesunken ist.
Weiter ist aufgefallen, dass der Anteil bei der Rechtswissenschaftlichen Fakultät im Sommersemester 2022 stark angestiegen ist. Dies lässt sich dadurch erklären, dass zu dieser Zeit das Juridicum fertiggestellt und eröffnet wurde.
Zu der Medizinischen Fakultät lässt sich sagen, dass der Anteil im Wintersemester 2008 plötzlich angestiegen ist. Diese haben auch erst vier Jahre nach den anderen Fakultäten angefangen, den entsprechenden Eintrag im UnivIS zu pflegen.
Als allgemeiner Trend ist zu beobachten, dass zu Beginn der Anteil steigt, bis ein Wert von ca. 50 % erreicht ist, dann bis Corona gleichbleibend ist und nach den Coronasemestern wieder auf ca. 50% ansteigt.

## Wie hat sich die Arbeitsbelastung einer Lehrperson bezüglich Lehrveranstaltungen entwickelt?

Wir haben uns für einen Lineplot entschieden, um die Entwicklung über die Zeit anzusehen. Die Arbeitsbelastung wird dabei angegeben als die Anzahl der durchschnittlichen Vorlesungen pro Lehrperson.

In [ ]:
fig = plotly.express.line(df, x="Semester", y="Arbeitsbelastung", color="Fakultät", title='', color_discrete_map=FACULTY_COLORS)
fig.update_traces(hovertemplate='<br>'.join([
    'Arbeitsbelastung: %{y:.2f} Durchschnittliche Vorlesungen pro Lehrperson'
]))
fig.update_layout(hovermode='x')

Die Arbeitsbelastung einer Lehrperson der medizinischen Fakultät ist von 4,56 durchschnittlichen Veranstaltungen pro Lehrperson auf 1,8 stark gesunken. Wenn sich die Arbeitsbelastung einer Fakultät erhöht hat, dann lediglich um höchstens eine Veranstaltung im untersuchten Zeitraum. Sonst ist aufgefallen, dass die Arbeitsbelastung bei manchen Fakultäten davon abhängig ist, ob gerade Sommer- oder Wintersemester ist. Zum Beispiel bei der Technischen Fakultät war ab dem Sommersemester 2013 die Arbeitsbelastung im Winter höher als im Sommer. Dies kehrte sich im Wintersemester 2021 um, was unter anderem auf die neue Fachprüfungsordnung der Informatik zurückzuführen sein könnte.
Insgesamt ist die Arbeitsbelastung über die Jahre hinweg relativ konstant geblieben.

## In welcher Fakultät finden die meisten Veranstaltungen auf Englisch statt?

Hier kann einfach die Funktion `visualize(df)` aus dem Modul `english.py` aufgerufen werden, um unsere Graphen anzeigen zu lassen. Wir haben uns für animierte Balkendiagramme bzw. für einen animierten Scatterplot entschieden.

In [ ]:
absolute, ratio_bar = english.visualize(df_english)

Durch die Variable „absolute“ wird ein Balkendiagramm angezeigt, dass die absolute Anzahl an Veranstaltungen auf Englisch visualisiert.

In [ ]:
absolute

 Der nächste Graph ist ein Balkendiagramm, der den prozentualen Anteil der englischsprachigeny Veranstaltungen pro Fakultät anzeigt.

In [ ]:
ratio_bar

Insgesamt ist zu sehen, dass die absolute Anzahl an englischen Modulen bei allen Fakultäten steigt. Eine Ausnahme bildet die Philosophische Fakultät.
Weiter ist auffällig, dass die Technische Fakultät seit 2009 im Wintersemester mehr englische Module anbietet als im Sommersemester. Im Wintersemester 2019 ist außerdem ein plötzlicher Anstieg zu sehen, der möglicherweise mit der Einführung der neuen Fachprüfungsordnung der Informatik zusammenhängen könnte.

Die Darstellung des Anteils der englischsprachigen Veranstaltungen bestätigt die vorherige Auswertung. Ansonsten ist zu erkennen, dass der Anteil sich bei den meisten Fakultäten auf unter 25% beschränkt, lediglich die Technische Fakultät nähert sich stetig den 50% an.

Anscheinend bieten die Medizinische und Theologische Fakultät keine englischen Module oder pflegen den entsprechenden Eintrag im UnivIS für die Modulsprache nicht.

## Wie hat sich der Anteil von Mitarbeitenden mit nicht-männlich erkannten Vornamen entwickelt?

 Die Theologische Fakultät hat sich am meisten entwickelt: Der Anteil von Mitarbeitenden mit nicht-männlich erkannten Vornamen startet im Sommersemester 2000 bei ca. 21% und liegt im Wintersemester 2022/23 bei ca. 54%.
 Die Medizinische Fakultät befindet sich seit dem Wintersemester 2006 immer über 50%. Die Technische Fakultät erreicht hingegen nicht einmal einen Wert von 30%.
Die Anteile der anderen Fakultäten steigen stetig, aber bleiben relativ konstant. Dabei erreicht keine Fakultät die 60%-Marke.
(Diese Auswertung bezieht sich auf eine Wahrscheinlichkeit einer richtigen Klassifizierung von 75%.)

## Gibt es Kriterien, anhand derer der Gewinner des Best-Prof-Awards vorhergesagt werden kann?